# Rewrite Basic SQL Queries in Pandas

Topic covered:

* Read the `titles.csv` into Jupyter Notebook
* Convert a simple SQL query (SELECT / FROM / WHERE / ORDER by) into Pandas query.
* Illustrate steps to refactor the Pandas query into more modular form
* Convert a simple SQL aggregation query (SELECT / COUNT(`*`) / GROUP BY / HAVING / ORDER BY) into Pandas query
* Wrap up: export SQL query output as CSV files

In [1]:
# Import the standard scientific libraries
import pandas as pd
import numpy as np

In [2]:
# Read CSV into a Pandas DataFrame
df = pd.read_csv("../input/netflix-tv-shows-and-movies/titles.csv")

So far we have used the default parameters when reading the CSV with `pd.read_csv()`. This is fine for initial analysis. You can however be more prescrptive such as manually defining column names, column data-types, delimiter, and more. If you are curious please refer to: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [3]:
type(df)

pandas.core.frame.DataFrame

In [4]:
# Take a peek at the top 5 rows 
df.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


Notice the columns with `NaN` (Not a number). This tends to happen when the column is numeric (`np.int32`, `np.int64`, `np.float32`, `np.float64`, etc.), and when there is no value for that column (known as Null in most programming languages). Had we imported this column as a Pandas `object` instead of `NaN` we would have have the Python `None` type.

In general, null values are stored differently depending on the data-type (`dtype`) of the column:

* For`object` column, null are stored as `None`.
* For numeric columns (like `np.int32`, `np.int64`, `np.float32`, `np.float64`, etc.), null are stored as `NaN`
* For other columns... (we can find out via documantation / experimentations). 

Not that important for now, but worth knowing...

Try this: on the right hand panel (Input section), expand "netflix-tv-shows-and-titles". Click on `titles.csv` to do a quick preview of what the raw data looks like. Notice the empty cells?

In [5]:
# What data-types have these columns been imported as?
df.dtypes

id                       object
title                    object
type                     object
description              object
release_year              int64
age_certification        object
runtime                   int64
genres                   object
production_countries     object
seasons                 float64
imdb_id                  object
imdb_score              float64
imdb_votes              float64
tmdb_popularity         float64
tmdb_score              float64
dtype: object

In [6]:
# Give me a bit more high level info for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

In [7]:
# What is the shape of the dataframe? (num_rows, num_columns)
df.shape

(5850, 15)

In [8]:
# print basic statistics
df.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,5850.000000,5850.000000,2106.000000,5368.000000,5.352000e+03,5759.000000,5539.000000
mean,2016.417094,76.888889,2.162868,6.510861,2.343938e+04,22.637925,6.829175
std,6.937726,39.002509,2.689041,1.163826,9.582047e+04,81.680263,1.170391
min,1945.000000,0.000000,1.000000,1.500000,5.000000e+00,0.009442,0.500000
25%,2016.000000,44.000000,1.000000,5.800000,5.167500e+02,2.728500,6.100000
50%,2018.000000,83.000000,1.000000,6.600000,2.233500e+03,6.821000,6.900000
75%,2020.000000,104.000000,2.000000,7.300000,9.494000e+03,16.590000,7.537500
max,2022.000000,240.000000,42.000000,9.600000,2.294231e+06,2274.044000,10.000000


# SQL Query Order of Execution

First of all, recall the SQL Query Order of Execution. Pandas syntax follows (**more or less** this order (with big caveat)). When we want to translate a SQL query into Pandas syntax, keep this in your head.

```
FROM
ON
JOIN
WHERE
GROUP BY
WITH CUBE or WITH ROLLUP
HAVING
SELECT
DISTINCT
ORDER BY
TOP or LIMIT
```

https://stackoverflow.com/questions/4596467/order-of-execution-of-the-sql-query

# SQL 1

In English:

* Show me the `type`, `release_year`, `runtime`, `title`
* from the table `df` ...
* where `type` is 'SHOW' AND release_year is greater than 2000
* sort by descending `year`, descending `runtime`, and ascending `title`

SQL Syntax:

```
SELECT type, release_year, runtime, title
FROM df
WHERE type = 'SHOW' AND release_year > 2000
ORDER BY release_year DESC, run_time DESC, title
;
```

SQL Order of execution

```
FROM df
WHERE type = 'SHOW' AND release_year > 2000
SELECT type, release_year, runtime, title
ORDER BY release_year DESC, run_time DESC, title
```

Let's try write the Pandas equivalent query (in SQL Order of execution)

### SQL 1 - Pandas Syntax - version 1

FROM `df`

... then `.loc[rows, columns]` to get the row slices that satisfy the `rows`, for the list of `columns`.

... then `.sort_values(by=[...], ascending=[...]).reset_index(drop=True)` to do that ORDER BY.


In [9]:
df\
.loc[((df["type"] == "SHOW") & (df["release_year"] > 2000)), :]\
.loc[:, ["type", "release_year", "runtime", "title"]]\
.sort_values(by=["release_year", "runtime", "title"], ascending=[False, False, True])\
.reset_index(drop=True)

# .reset_index(drop=True)

,type,release_year,runtime,title
0,SHOW,2022,93,jeen-yuhs
1,SHOW,2022,92,Jump Like a Witch
2,SHOW,2022,85,Jimmy Savile: A British Horror Story
3,SHOW,2022,80,Alchemy of Souls
4,SHOW,2022,76,Twenty Five Twenty One
...,...,...,...,...
2061,SHOW,2001,48,El Escamoso
2062,SHOW,2001,47,The Mole
2063,SHOW,2001,25,Trailer Park Boys
2064,SHOW,2001,23,The Fairly OddParents


Technically, we can make the Pandas query shorter!

### SQL 1 - Pandas Syntax - version 2

SQL Syntax:

```
SELECT *

FROM (
    SELECT type, release_year, runtime, title
    FROM df
    WHERE type = 'SHOW' AND release_year > 2000
) AS a

ORDER BY a.release_year DESC, a.run_time DESC, a.title
;
```

SQL Order of execution

```
FROM (
    SELECT type, release_year, runtime, title
    FROM df
    WHERE type = 'SHOW' AND release_year > 2000
) AS a

SELECT *

ORDER BY a.release_year DESC, a.run_time DESC, a.title
```

In [10]:
# Before:
# df\
# .loc[((df["type"] == "SHOW") & (df["release_year"] > 2000)), :]\
# .loc[:, ["type", "release_year", "runtime", "title"]]\
# .sort_values(by=["release_year", "runtime", "title"], ascending=[False, False, False])

# After:
df\
.loc[((df["type"] == "SHOW") & (df["release_year"] > 2000)), ["type", "release_year", "runtime", "title"]]\
.sort_values(by=["release_year", "runtime", "title"], ascending=[False, False, True]).reset_index(drop=True)

,type,release_year,runtime,title
0,SHOW,2022,93,jeen-yuhs
1,SHOW,2022,92,Jump Like a Witch
2,SHOW,2022,85,Jimmy Savile: A British Horror Story
3,SHOW,2022,80,Alchemy of Souls
4,SHOW,2022,76,Twenty Five Twenty One
...,...,...,...,...
2061,SHOW,2001,48,El Escamoso
2062,SHOW,2001,47,The Mole
2063,SHOW,2001,25,Trailer Park Boys
2064,SHOW,2001,23,The Fairly OddParents


### SQL 1 - Pandas Syntax - version 3 (Modular)

In [11]:
# Before:
# df\
# .loc[((df["type"] == "SHOW") & (df["release_year"] > 2000)), ["type", "release_year", "runtime", "title"]]\
# .sort_values(by=["release_year", "runtime", "title"], ascending=[False, False, False])

# After:
my_where = ((df["type"] == "SHOW") & (df["release_year"] > 2000))

my_select = ["type", "release_year", "runtime", "title"]

my_sort = {
    "by": ["release_year", "runtime", "title"],
    "ascending": [False, False, True]
}

df\
.loc[my_where, my_select]\
.sort_values(**my_sort).reset_index(drop=True)

,type,release_year,runtime,title
0,SHOW,2022,93,jeen-yuhs
1,SHOW,2022,92,Jump Like a Witch
2,SHOW,2022,85,Jimmy Savile: A British Horror Story
3,SHOW,2022,80,Alchemy of Souls
4,SHOW,2022,76,Twenty Five Twenty One
...,...,...,...,...
2061,SHOW,2001,48,El Escamoso
2062,SHOW,2001,47,The Mole
2063,SHOW,2001,25,Trailer Park Boys
2064,SHOW,2001,23,The Fairly OddParents


In [12]:
# quick illustration of what dictionary unpacking look like
{**my_sort}

{'by': ['release_year', 'runtime', 'title'], 'ascending': [False, False, True]}

In [13]:
# quick illustration of our pandas query in one liner
df.loc[my_where, my_select].sort_values(**my_sort).reset_index(drop=True)

,type,release_year,runtime,title
0,SHOW,2022,93,jeen-yuhs
1,SHOW,2022,92,Jump Like a Witch
2,SHOW,2022,85,Jimmy Savile: A British Horror Story
3,SHOW,2022,80,Alchemy of Souls
4,SHOW,2022,76,Twenty Five Twenty One
...,...,...,...,...
2061,SHOW,2001,48,El Escamoso
2062,SHOW,2001,47,The Mole
2063,SHOW,2001,25,Trailer Park Boys
2064,SHOW,2001,23,The Fairly OddParents


In [14]:
# What does my_where look like?
# my_where = ((df["type"] == "SHOW") & (df["release_year"] > 2000))
my_where

0       False
1       False
2       False
3       False
4       False
        ...  
5845    False
5846    False
5847    False
5848    False
5849     True
Length: 5850, dtype: bool

In [15]:
# show all rows in df
print(f"total rows in df: {len(df)}")

# show all rows in df that satisfy the my_where condition
print(f"total rows in df that satisfy the my_where condition: {len(df[my_where])}")

# show all rows in df that do not satisfy the my_where condition
print(f"total rows in df that do not satisfy the my_where condition: {len(df[~my_where])}")

# the numbers always add up: total rows in df == total rows that satisfy conditions + total rows that do not satisfy conditions 
assert len(df) == len(df[my_where]) + len(df[~my_where])

total rows in df: 5850
total rows in df that satisfy the my_where condition: 2066
total rows in df that do not satisfy the my_where condition: 3784


# SQL 2 - Group By

SQL Syntax:
    
```
SELECT release_year, COUNT(*) AS title_count
FROM df
GROUP BY release_year
HAVING count(*) < 10
ORDER BY release_year DESC, run_time DESC, title
```

SQL Order of execution:

```
FROM df
GROUP BY release_year
HAVING count(*) < 10
SELECT release_year, COUNT(*) AS title_count
ORDER BY release_year DESC
;
```

Pandas order of execution:

```
FROM df
GROUP BY release_year
    COUNT(*) AS title_count
```

```
HAVING title_count < 10
SELECT release_year, title_count
ORDER BY release_year DESC
;
```

In [16]:
df\
.groupby(by=["release_year"])\
.size().reset_index(name='title_count')

,release_year,title_count
0,1945,1
1,1954,2
2,1956,1
3,1958,1
4,1959,1
...,...,...
58,2018,773
59,2019,836
60,2020,814
61,2021,787


In [17]:
df1 = df\
.groupby(by=["release_year"])\
.size()\
.reset_index(name='title_count')

In [18]:
df1

,release_year,title_count
0,1945,1
1,1954,2
2,1956,1
3,1958,1
4,1959,1
...,...,...
58,2018,773
59,2019,836
60,2020,814
61,2021,787


In [19]:
df1\
.loc[df1["title_count"] < 10, :]\
.sort_values(by=["title_count"], ascending=[True]).reset_index(drop=True)

,release_year,title_count
0,1945,1
1,1985,1
2,1956,1
3,1958,1
4,1959,1
5,1960,1
6,1961,1
7,1963,1
8,1966,1
9,1978,1


There are more to explore and experiment with Pandas aggregation queries. For next time!

Google Pandas + your query description

## Wrap up: export CSVs

Our goal here is to export the query output (Pandas DataFrames) as CSV files, to the Kaggle working directory `/kaggle/working`. We may then download to our laptop as needed. (Note: purely for demo sake!)

Note: if you go to the right hand side pandel (Output section), you may copy the file path for `/kaggle/working`. It would resolve to `./` (Because our current work directory is `/kaggle/working`). Use this `./` as our folder path for CSV export.

We will use the default parameters for `pd.to_csv()` for now. If you are curious please refer to: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

### SQL 1 - to CSV

```
SELECT type, release_year, runtime, title
FROM df
WHERE type = 'SHOW' AND release_year > 2000
ORDER BY release_year DESC, run_time DESC, title
;
```

In [20]:
my_where = ((df["type"] == "SHOW") & (df["release_year"] > 2000))
my_select = ["type", "release_year", "runtime", "title"]
my_sort = {
    "by": ["release_year", "runtime", "title"],
    "ascending": [False, False, False]
}

df_sql_1 = df\
.loc[my_where, my_select]\
.sort_values(**my_sort).reset_index(drop=True)

# print first 5 rows to console
print(df_sql_1.shape)
print(df_sql_1.dtypes)
print(df_sql_1.head(5))

# export the whole dataset to CSV file
df_sql_1.to_csv("./df_sql_1.csv")

(2066, 4)
type            object
release_year     int64
runtime          int64
title           object
dtype: object
   type  release_year  runtime                                 title
0  SHOW          2022       93                             jeen-yuhs
1  SHOW          2022       92                     Jump Like a Witch
2  SHOW          2022       85  Jimmy Savile: A British Horror Story
3  SHOW          2022       80                      Alchemy of Souls
4  SHOW          2022       76                Twenty Five Twenty One


### SQL 2 - to CSV
    
```
SELECT release_year, COUNT(*) AS title_count
FROM df
GROUP BY release_year
HAVING count(*) < 10
ORDER BY release_year DESC, run_time DESC, title
```

In [21]:
_df_sql_2 = df\
.groupby(by=["release_year"])\
.size()\
.reset_index(name='title_count')

df_sql_2 = _df_sql_2\
.loc[_df_sql_2["title_count"] < 10]\
.sort_values(by=["title_count"], ascending=[True]).reset_index(drop=True)

# print first 5 rows to console
print(df_sql_2.shape)
print(df_sql_2.dtypes)
print(df_sql_2.head(5))

# export the whole dataset to CSV file
df_sql_2.to_csv("./df_sql_2.csv")

(32, 2)
release_year    int64
title_count     int64
dtype: object
   release_year  title_count
0          1945            1
1          1985            1
2          1956            1
3          1958            1
4          1959            1


If you go to the right hand panel Output section now, you should see `df_sql_1.csv` and `df_sql_2.csv2`. Take a look. Or download to your laptop and open.